In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# --- Summary model (русский mBART) ---
sum_model_name = "IlyaGusev/mbart_ru_sum_gazeta"
sum_tokenizer = AutoTokenizer.from_pretrained(sum_model_name)
sum_model = AutoModelForSeq2SeqLM.from_pretrained(sum_model_name)

def generate_summary(text):
    input_ids = sum_tokenizer([text], return_tensors="pt", max_length=1024, truncation=True).input_ids
    output_ids = sum_model.generate(
        input_ids=input_ids,
        num_beams=4,
        max_length=200,
        min_length=30,
        repetition_penalty=1.2,
        early_stopping=True
    )
    return sum_tokenizer.decode(output_ids[0], skip_special_tokens=True)

# --- Sentiment model (русский) ---
sentiment_pipeline = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment")

def detect_sentiment(text):
    result = sentiment_pipeline(text[:512])[0]
    return result['label'].lower()

# --- Tone classifier ---
tone_labels = ["факт", "аналитика", "мнение", "интервью", "расследование"]
tone_pipeline = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")

def detect_tone(text):
    result = tone_pipeline(text[:512], tone_labels)
    return result["labels"][0]

# --- Named Entity Recognition ---
ner_pipeline = pipeline("ner", model="Davlan/xlm-roberta-base-ner-hrl", aggregation_strategy="simple")

zero_shot = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")

CATEGORY_LABELS = [
    "политика",
    "экономика",
    "бизнес",
    "технологии",
    "спорт",
    "общество",
    "происшествия",
    "культура",
    "наука",
    "авто",
    "медиа",
    "здоровье",
    "путешествия"
]

SUBCATEGORY_LABELS = {
    "политика": [
        "выборы", "санкции", "коррупция", "международные отношения", "госуправление", "законопроекты", "военные конфликты"
    ],
    "экономика": [
        "инфляция", "безработица", "налоги", "бюджет", "валюта", "госдолг", "макроэкономика"
    ],
    "бизнес": [
        "рынки", "сделки", "финансовая отчётность", "IPO", "корпорации", "предпринимательство", "банковский сектор"
    ],
    "технологии": [
        "искусственный интеллект", "стартапы", "гаджеты", "кибербезопасность", "социальные сети", "блокчейн", "робототехника"
    ],
    "спорт": [
        "футбол", "олимпийские игры", "теннис", "хоккей", "трансферы", "чемпионаты", "рекорды"
    ],
    "общество": [
        "образование", "миграция", "семья", "трудоустройство", "социальная политика", "молодёжь", "демография"
    ],
    "происшествия": [
        "пожары", "ДТП", "преступления", "наводнения", "теракты", "взрывы", "катастрофы"
    ],
    "культура": [
        "кино", "литература", "театр", "музыка", "живопись", "фестивали", "музеи"
    ],
    "наука": [
        "астрофизика", "биология", "медицина", "экология", "инновации", "исследования", "технологические открытия"
    ],
    "авто": [
        "электромобили", "дорожные тесты", "новые модели", "аварии", "автосалон", "тюнинг", "грузовики"
    ],
    "медиа": [
        "телевидение", "интернет-СМИ", "журналистика", "цензура", "социальные медиа", "трансляции", "инфлюенсеры"
    ],
    "здоровье": [
        "психология", "заболевания", "лекарства", "здоровый образ жизни", "медицина", "эпидемии", "медуслуги"
    ],
    "путешествия": [
        "туризм", "авиаперелёты", "отели", "визы", "круизы", "гиды", "страны и города"
    ]
}

def classify_category(text):
    result = zero_shot(text[:1000], candidate_labels=CATEGORY_LABELS)
    return result["labels"][0]

def classify_subtopics(text, category):
    candidates = SUBCATEGORY_LABELS.get(category, [])
    if not candidates:
        return []
    result = zero_shot(text[:1000], candidate_labels=candidates, multi_label=True)
    return [label for label, score in zip(result['labels'], result['scores']) if score > 0.4]

def extract_named_entities(text):
    ents = ner_pipeline(text[:1000])
    people, orgs, locs = set(), set(), set()
    for ent in ents:
        if ent['entity_group'] == 'PER':
            people.add(ent['word'])
        elif ent['entity_group'] == 'ORG':
            orgs.add(ent['word'])
        elif ent['entity_group'] == 'LOC':
            locs.add(ent['word'])
    return {
        "persons": list(people),
        "orgs": list(orgs),
        "locations": list(locs)
    }

In [ ]:

for val in tqdm(d):
    if len(val["text"]) < 10:
        if len(val["title"]) < 10:
            val["summary"] = None
            val["sentiment"] = None
            val["tone"] = None
            val["named_entities"] = None
            val["category"] = None
            val["subtopics"] = None
            continue
        val["summary"] = generate_summary(val["title"])
        #val["sentiment"] = detect_sentiment(val["title"])
        #val["tone"] = detect_tone(val["title"])
        #val["named_entities"] = extract_named_entities(val["title"])
        #val["category"] = classify_category(val["title"])
        #val["subtopics"] = classify_subtopics(val["title"], val["category"])
        continue
    text = val["text"]
    val["summary"] = generate_summary(text)
    #val["sentiment"] = detect_sentiment(text)
    #val["tone"] = detect_tone(text)
    #val["named_entities"] = extract_named_entities(text)
    #val["category"] = classify_category(text)
    #val["subtopics"] = classify_subtopics(text, val["category"])

In [ ]:
import json
with open("res.json", "r") as f:
    data = json.loads(f.read())


DDD = {
    "negative": "Негативная",
    "neutral": "Нейтральная",
    "positive": "Позитивная"
}



In [ ]:
prompt = "На основе новостей ниже:\n" \
            "1. Обобщи основную информацию.\n" \
            "2. Укажи общие тренды или повторяющиеся темы.\n" \
            "3. Сделай это кратко — 3-4 предложения.\n" \
            "Не повторяй заголовки и не копируй слово в слово. Выдели главное.\n\n"

for i, news in enumerate(data[:5], 1):
    prompt += f"НОВОСТЬ {i}:\n"
    prompt += f"Заголовок: {news.get('title')}\n"
    prompt += f"Категория: {news.get('category')}\n"
    if news.get('subtopics'):
        prompt += f"Подкатегории: {', '.join(news.get('subtopics'))}\n"
    prompt += f"Резюме: {news.get('summary')}\n"
    prompt += f"Тональность: {news.get('tone')}, Оценка: {DDD[news['sentiment']]}\n\n"


In [ ]:
print(prompt)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "IlyaGusev/mbart_ru_sum_gazeta" # "tiiuae/falcon-rw-1b" #"mistralai/Mistral-7B-Instruct-v0.2"

# Загружаем токенизатор и модель
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map={"": "mps"}  # Используем Apple Metal (GPU)
)

# Создаем пайплайн для генерации текста
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map={"": "mps"},
    torch_dtype=torch.float16,
)


# Генерация
output = llm(prompt, max_new_tokens=300, do_sample=True, temperature=0.7)[0]["generated_text"]

# Печатаем только ответ после prompt
print("\n🔍 Вывод:")
print(output[len(prompt):].strip())


In [ ]:
from src.parser.driverv2 import process_news_batch
from concurrent.futures import ProcessPoolExecutor, as_completed
import asyncio

from src.utils.chunkify import chunkify


items = []
proxy = {
    "server": "http://123.45.67.89:8000",
    "username": "your_user",
    "password": "your_pass"
}

result = asyncio.run(process_news_batch(items, proxy=proxy))


# run buthces
def run_batch(items, proxy):
    return asyncio.run(process_news_batch(items, proxy=proxy))

max_workers = 3
num_chunks = 3
chunks = chunkify(news, num_chunks)

results = []
for i in range(0, len(chunks), max_workers):
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(run_batch, chunk, proxy) for chunk in chunks[i:i + max_workers]]
        for future in as_completed(futures):
            result = future.result() 
            results.extend(result)

In [ ]:
import requests

data = requests.post(
    url="http://localhost:11434/api/embeddings",
    json={
        "model": "nomic-embed-text",
        "prompt": "search_document: Привет как дела?",
    }
)

In [ ]:
res = data.json()
len(res["embedding"])

In [1]:
from redis import ConnectionPool, Redis
import json


pool = ConnectionPool.from_url(
    url="redis://localhost:6379/0",
    max_connections=5,
    socket_timeout=30,
    db=0,
    encoding="utf-8",
    socket_connect_timeout=20,
    health_check_interval=30,
    retry_on_timeout=True,
    decode_responses=True,
)
redis = Redis(connection_pool=pool)

In [2]:

import re

def clean_news(raw_text):
    # 1. Удаление дубликатов строк (оставим уникальные)
    lines = raw_text.splitlines()
    unique_lines = []
    seen = set()
    for line in lines:
        line = line.strip()
        if line and line not in seen:
            unique_lines.append(line)
            seen.add(line)

    # 2. Удаление ISO-дат (в формате 2025-04-19T16:22:00+03:00)
    clean_lines = [re.sub(r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\+\d{2}:\d{2}', '', line) for line in unique_lines]

    # 3. Удаление email, телефонов и чисел без контекста
    clean_lines = [re.sub(r'\S+@\S+|\d[\d\s-]{5,}|\d{3,}', '', line) for line in clean_lines]

    # 5. Удаление повторений в заголовках
    final_lines = []
    for line in clean_lines:
        if not final_lines or final_lines[-1] != line:
            final_lines.append(line)

    # 6. Объединение обратно в текст
    result = '\n'.join(final_lines)
    
    # 7. Удаление лишних пробелов
    result = re.sub(r'\n\s*\n', '\n\n', result).strip()
    return result


In [ ]:
import requests
from datetime import datetime, UTC


d = {
  "ru": (
      "Сделай краткое содержание текста, сохранив только ключевые факты, важные высказывания и значимые действия. "
      "Сохраняй политический и эмоциональный контекст. Приводи кратко, но ёмко." 
      "Ответ должен содержать только итоговое краткое содержание без пояснений и лишнего текста и его форматирования.\n\n"
  ),
  "en": (
      "Make a summary of the text, keeping only the key facts, important statements and significant actions. "
      "Keep the political and emotional context. Keep it brief but to the point." 
      "The answer should contain only a summary of the final content without explanations and unnecessary text and its formatting.\n\n"
  )
}
i = 0
s = len(news)
m = []
for a in news:
  start = datetime.now(UTC)
  print(datetime.now(UTC).strftime("[%H:%M:%S]"), a["lang"], f't len: {len(a["article"]["text"])}', end=" ")
  prompt = d[a["lang"]] + a["article"]["text"]
  resp = requests.post(
    url="http://localhost:11434/api/generate",
    json={
      "model": "llama3.2",
      "prompt": prompt,
      "stream": False,
    },
  )
  summary = resp.json()["response"]
  print(resp.status_code, f"s_len: {len(summary)}", f"{(datetime.now(UTC) - start).total_seconds()}", end=" ")

  resp2 = requests.post(
    url="http://localhost:8080/classify/labels",
    json={
      "text": summary,
      "lang": a["lang"]
    },
  )
  asn = resp2.json()
  print(resp2.status_code, f"{(datetime.now(UTC) - start).total_seconds()}", end=" ")
  subcategories = [i["label"] for i in asn["subcategories"]]
  news_item = {
      "title": a["article"]["title"],
      "content": a["article"]["text"],
      "summary": resp.json()["response"],
      "source_url": a["link"],
      "lang": a["lang"],
      "category": asn["category"],
      "subcategories": subcategories,
      "companies": [],
      "locations": [],
      "names": [],
      "priority": 0,
      "is_ready": True,
      "published_at": a["published"],
      "created_at": datetime.now(UTC).isoformat()
}
  result = requests.post(
    url="http://localhost:8000/news",
    json=news_item
  )
  end = datetime.now(UTC)
  delta = (end - start).total_seconds()
  i += 1
  print(result.status_code, f"{i}/{s}: {delta}")
  m.append((delta, len(a["article"]["text"]), len(summary), a["lang"]))
  

In [3]:
news = []
for m in redis.xrange("raw_news"):
    key = m[-1]["key"]
    res = json.loads(redis.get(key))
    news.extend(res)

# last news: https://www.yahoo.com/news/vatican-postpones-sainthood-gods-influencer-112236924.html

In [4]:
news[0]

{'link': 'https://www.championat.com/football/article-5981244-barselona-selta-4-3-obzor-matcha-32-go-tura-la-ligi-goly-ferran-torres-iglesias-rafinya-statistika-19-aprelya-2025.html',
 'published': '2025-04-19T19:42:53+03:00',
 'lang': 'ru',
 'success': True,
 'article': {'source_url': 'https://www.championat.com',
  'url': 'https://www.championat.com/football/article-5981244-barselona-selta-4-3-obzor-matcha-32-go-tura-la-ligi-goly-ferran-torres-iglesias-rafinya-statistika-19-aprelya-2025.html',
  'title': 'Барселона — Сельта — 4:3, обзор матча 32-го тура Ла Лиги, голы: Ферран Торрес, Иглесиас, Рафинья, статистика, 19 апреля 2025',
  'top_img': 'https://img.championat.com/news/social/0/58/5981244.jpg',
  'top_image': 'https://img.championat.com/news/social/0/58/5981244.jpg',
  'meta_img': 'https://img.championat.com/news/social/0/58/5981244.jpg',
  'images': "{'https://counter.yadro.ru/logo;championat_com?52.3', 'https://img.championat.com/team/logo/14679046901516426008.png', 'https://

In [35]:
news2 = []
f = False
for n in news:
    if f:
        news2.append(n)
    if n["link"] == "https://www.yahoo.com/news/vatican-postpones-sainthood-gods-influencer-112236924.html":
        f = True

len(news2)

1163

In [ ]:
news[-1]["link"]

In [ ]:
from collections import Counter
from urllib.parse import urlparse

m = Counter([urlparse(n["article"]["source_url"]).netloc for n in news])



In [ ]:
print(news[0]["link"])
print(news[0]["article"]["text"])

In [36]:
import requests
from datetime import datetime, UTC
from tqdm import tqdm

for a in tqdm(news2):
  start = datetime.now(UTC)
  news_item = {
      "title": a["article"]["title"],
      "content": a["article"]["text"],
      "summary": "",
      "source_url": a["link"],
      "lang": a["lang"],
      "category": "",
      "subcategories": [],
      "companies": [],
      "locations": [],
      "names": [],
      "priority": 0,
      "is_ready": False,
      "published_at": a["published"],
      "created_at": datetime.now(UTC).isoformat()
}
  result = requests.post(
    url="http://localhost:8000/news",
    json=news_item
  )
  

100%|██████████| 1163/1163 [00:03<00:00, 378.08it/s]


In [1]:
from typing import List, Optional, Union, Literal
from pydantic import BaseModel, Field
import requests


class Message(BaseModel):
    role: Literal["system", "user", "assistant", "developer"]
    content: str


class KBFilter(BaseModel):
    index: str
    path: Optional[str] = None


class ChatCompletionRequest(BaseModel):
    messages: List[Message]
    temperature: Optional[float] = Field(default=None, ge=0.0, le=2.0)
    top_p: Optional[float] = Field(default=None, ge=0.01)
    max_tokens: Optional[int] = Field(default=None, ge=1)
    max_completion_tokens: Optional[int] = Field(default=None, ge=1)
    stream: Optional[bool] = False
    k: Optional[int] = Field(default=None, ge=0)
    retrieval_method: Optional[Literal["rewrite", "step_back", "sub_queries", "none"]] = None
    frequency_penalty: Optional[float] = Field(default=0.0, ge=-2.0, le=2.0)
    presence_penalty: Optional[float] = Field(default=0.0, ge=-2.0, le=2.0)
    stop: Optional[Union[str, List[str]]] = None
    instruction_override: Optional[str] = None
    include_functions_info: Optional[bool] = False
    include_retrieval_info: Optional[bool] = False
    include_guardrails_info: Optional[bool] = False
    provide_citations: Optional[bool] = False
    kb_filters: Optional[List[KBFilter]] = None
    filter_kb_content_by_query_metadata: Optional[bool] = False


class GenAIAgentClient:
    def __init__(self, base_url: str, api_key: str):
        self.base_url = base_url.rstrip('/')
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }

    def health_check(self) -> dict:
        url = f"{self.base_url}/health"
        response = requests.get(url, headers=self.headers)
        response.raise_for_status()
        return response.json()

    def chat_completion(self, payload: ChatCompletionRequest) -> dict:
        url = f"{self.base_url}/api/v1/chat/completions"
        response = requests.post(url, headers=self.headers, json=payload.model_dump(exclude_none=True))
        response.raise_for_status()
        return response.json()


In [2]:
client = GenAIAgentClient(base_url="https://bm7tmpgxhhbq42xfwdsvlzjf.agents.do-ai.run/", api_key="r3U2WKvdPenZ3hX9dK29tqFIcTyQpl2n")


In [3]:
client.health_check()

{'status': 'ok'}

In [41]:
client = GenAIAgentClient(base_url="https://bm7tmpgxhhbq42xfwdsvlzjf.agents.do-ai.run/", api_key="r3U2WKvdPenZ3hX9dK29tqFIcTyQpl2n")

request_payload = ChatCompletionRequest(
    messages=[
        Message(role="system", content="You are a news analyst assistant."),
        Message(role="user", content=news[0]["article"]["text"])
    ],
    temperature=0.3,
    top_p=0.9,
    max_tokens=3000,
    max_completion_tokens=400,
)

response = client.chat_completion(payload=request_payload)

In [42]:
response["choices"][0]["message"]["content"]

'«Барселона» совершила камбэк в матче против «Сельты», проигрывая 1:3, но в итоге выиграв 4:3. Борха Иглесиас забил хет-трик за «Сельту», но «Барселона» смогла отыграться благодаря голам Дани Ольмо, Рафиньи и Ламина. Победный гол был забит Рафиньей на 90+6-й минуте с пенальти. Эта победа позволила «Барселоне» увеличить преимущество над «Реалом» в чемпионате до 7 очков.'

In [39]:
response["usage"]

{'prompt_tokens': 2166, 'completion_tokens': 169, 'total_tokens': 2335}

In [73]:
import requests
from datetime import datetime, UTC

d = {
  "ru": (
      "Ты аналитик новостей\n"
      "Сделай краткое содержание текста, сохранив только ключевые факты, важные высказывания и значимые действия. "
      "Сохраняй политический и эмоциональный контекст. Приводи кратко, но ёмко." 
      "Ответ должен содержать только итоговое краткое содержание без пояснений и лишнего текста и его форматирования.\n\n"
  ),
  "en": (
      "You are a news analyst assistant.\n"
      "Make a summary of the text, keeping only the key facts, important statements and significant actions. "
      "Keep the political and emotional context. Keep it brief but to the point." 
      "The answer should contain only a summary of the final content without explanations and unnecessary text and its formatting.\n\n"
  )
}
client = GenAIAgentClient(base_url="https://bm7tmpgxhhbq42xfwdsvlzjf.agents.do-ai.run/", api_key="r3U2WKvdPenZ3hX9dK29tqFIcTyQpl2n")

limit = 10
offset = 0
results = []
while True:
  news_items_raw = requests.get(url=f"http://0.0.0.0:8000/news?limit={limit}&offset={offset}")
  news_items = news_items_raw.json()
  offset += limit
  if not news_items:
    break
  
  for obj in news_items:
    if len(obj["content"].replace(" ", "")) < 50:
      continue
    request_payload = ChatCompletionRequest(
        messages=[Message(role="system", content=d[obj["lang"]]), Message(role="user", content=obj["content"])],
        temperature=0.3,
        top_p=0.9,
        max_tokens=2500,
        max_completion_tokens=400,
    )

    start = datetime.now(UTC)
    response = client.chat_completion(payload=request_payload)  
    delta = (datetime.now(UTC) - start).total_seconds()
    summary = response["choices"][0]["message"]["content"]
    result = requests.put(
      url=f"http://localhost:8000/news?id={obj['id']}",
      json={"summary": summary}
    )
    results.append(
      {
        "article_size": len(obj["content"]),
        "summary_size": len(summary),
        "prompt_tokens": response["usage"]["prompt_tokens"],
        "completion_tokens": response["usage"]["completion_tokens"],
        "total_tokens": response["usage"]["total_tokens"],
        "status": result.status_code,
        "lang": obj["lang"],
        "delta": delta,
        "id": obj['id'],
      }
    )
  
  

HTTPError: 500 Server Error: Internal Server Error for url: https://bm7tmpgxhhbq42xfwdsvlzjf.agents.do-ai.run/api/v1/chat/completions

In [74]:
import pandas as pd

df = pd.DataFrame(results)
df.head()

,article_size,summary_size,prompt_tokens,completion_tokens,total_tokens,status,lang,delta,id
0,1539,436,1282,197,1479,200,ru,6.115160,8
1,3489,425,2164,194,2358,200,ru,9.551833,9
2,7135,605,3864,276,4140,200,ru,7.379068,11
3,1593,634,1316,289,1605,200,ru,8.584833,14
4,2533,232,1747,104,1851,200,ru,4.001243,2


In [ ]:

df.mean(0, numeric_only=True)

article_size         2517.905263
summary_size          418.045113
prompt_tokens        1495.457895
completion_tokens     154.283459
total_tokens         1649.741353
status                200.000000
delta                   4.533099
id                    980.612030
dtype: float64

In [97]:
df.lang.value_counts()["ru"]

np.int64(927)

In [ ]:
import numpy as np

p = 80

ddd = df[df.lang == "ru"]
print(f"percentile {p}\n")
print(f"RU count {df.lang.value_counts()['ru']}")
article_size = np.percentile(ddd.article_size, p)
print(f"article_size: {article_size}")

summary_size = np.percentile(ddd.summary_size, p)
print(f"summary_size: {summary_size}")

prompt_tokens = np.percentile(ddd.prompt_tokens, p)
print(f"prompt_tokens: {prompt_tokens}")

completion_tokens = np.percentile(ddd.completion_tokens, p)
print(f"completion_tokens: {completion_tokens}")

total_tokens = np.percentile(ddd.total_tokens, p)
print(f"total_tokens: {total_tokens}")

delta = np.percentile(ddd.delta, p)
print(f"delta: {delta}")

print("-" * 10)

ddd = df[df.lang == "en"]
print(f"EN count {df.lang.value_counts()['en']}")
article_size = np.percentile(ddd.article_size, p)
print(f"article_size: {article_size}")

summary_size = np.percentile(ddd.summary_size, p)
print(f"summary_size: {summary_size}")

prompt_tokens = np.percentile(ddd.prompt_tokens, p)
print(f"prompt_tokens: {prompt_tokens}")

completion_tokens = np.percentile(ddd.completion_tokens, p)
print(f"completion_tokens: {completion_tokens}")

total_tokens = np.percentile(ddd.total_tokens, p)
print(f"total_tokens: {total_tokens}")

delta = np.percentile(ddd.delta, p)
print(f"delta: {delta}")



print(f"total price: {1.48 - 0.05}")


percentile 80

RU count 927
article_size: 2395.8
summary_size: 475.80000000000007
prompt_tokens: 1672.0
completion_tokens: 217.0
total_tokens: 1882.8000000000002
delta: 5.840963200000002
----------
EN count 403
article_size: 4901.4
summary_size: 731.8000000000001
prompt_tokens: 1762.8000000000002
completion_tokens: 182.60000000000002
total_tokens: 1951.8000000000002
delta: 5.591718600000001
total price: 1.43


In [101]:
ddd = df
article_size = np.percentile(ddd.article_size, p)
print(f"article_size: {article_size}")

summary_size = np.percentile(ddd.summary_size, p)
print(f"summary_size: {summary_size}")

prompt_tokens = np.percentile(ddd.prompt_tokens, p)
print(f"prompt_tokens: {prompt_tokens}")

completion_tokens = np.percentile(ddd.completion_tokens, p)
print(f"completion_tokens: {completion_tokens}")

total_tokens = np.percentile(ddd.total_tokens, p)
print(f"total_tokens: {total_tokens}")

delta = np.percentile(ddd.delta, p)
print(f"delta: {delta}")

article_size: 3382.6000000000004
summary_size: 583.0
prompt_tokens: 1712.0
completion_tokens: 206.20000000000005
total_tokens: 1902.0
delta: 5.7708144


In [105]:
total_price = 1.48 - 0.05
price_per_arcticle = total_price / df.shape[0]
price_per_arcticle * 1000 * 31

33.33082706766917

In [106]:
df.to_csv("expore.csv")

In [7]:
content = '```json\n{\n  "category": "спорт",\n  "subcategories": ["футбол", "чемпионат", "камбэк"],\n  "importance": 2,\n  "summary": "Футбольная команда «Барселона» совершила камбэк в матче против «Сельты», проигрывая 1:3, но в итоге выиграв 4:3. «Барселона» продемонстрировала чемпионский характер и увеличила преимущество над «Реалом» в чемпионате до 7 очков.",\n  "companies": [],\n  "locations": ["Виго", "Дортмунд"],\n  "names": ["Ханси Флик", "Бальде", "Араухо", "Гави", "Левандовски", "Фермин", "Гуайта", "Ферран", "Алонсо", "Дуран", "Кубарси", "Щенсны", "Иглесиас", "Войцех", "Мориба", "Лосады", "Мартин", "Бальде", "Де Йонг", "Мартинес", "Лаго", "Каррейра", "Ольмо", "Ямал", "Рафинья", "Ламин", "Лопес", "Мингеса"]\n}\n```'
content = content.replace("`", "")
content = content.replace("json", "")
content = content.strip("\n")
content = content.strip(" ")
print(content)

{
  "category": "спорт",
  "subcategories": ["футбол", "чемпионат", "камбэк"],
  "importance": 2,
  "summary": "Футбольная команда «Барселона» совершила камбэк в матче против «Сельты», проигрывая 1:3, но в итоге выиграв 4:3. «Барселона» продемонстрировала чемпионский характер и увеличила преимущество над «Реалом» в чемпионате до 7 очков.",
  "companies": [],
  "locations": ["Виго", "Дортмунд"],
  "names": ["Ханси Флик", "Бальде", "Араухо", "Гави", "Левандовски", "Фермин", "Гуайта", "Ферран", "Алонсо", "Дуран", "Кубарси", "Щенсны", "Иглесиас", "Войцех", "Мориба", "Лосады", "Мартин", "Бальде", "Де Йонг", "Мартинес", "Лаго", "Каррейра", "Ольмо", "Ямал", "Рафинья", "Ламин", "Лопес", "Мингеса"]
}


In [8]:
from pydantic import BaseModel, Field

class LLMResp(BaseModel):
    category: str = Field(description="category")
    subcategories: list[str] = Field(description="subcategories")
    summary: str = Field(description="summary")
    importance: int = Field(description="importance")
    companies: list[str] = Field(description="companies", default=[])
    locations: list[str] = Field(description="locations", default=[])
    names: list[str] = Field(description="names", default=[])

In [11]:
LLMResp.model_validate_json(content.replace("\n", ""))

LLMResp(category='спорт', subcategories=['футбол', 'чемпионат', 'камбэк'], summary='Футбольная команда «Барселона» совершила камбэк в матче против «Сельты», проигрывая 1:3, но в итоге выиграв 4:3. «Барселона» продемонстрировала чемпионский характер и увеличила преимущество над «Реалом» в чемпионате до 7 очков.', importance=2, companies=[], locations=['Виго', 'Дортмунд'], names=['Ханси Флик', 'Бальде', 'Араухо', 'Гави', 'Левандовски', 'Фермин', 'Гуайта', 'Ферран', 'Алонсо', 'Дуран', 'Кубарси', 'Щенсны', 'Иглесиас', 'Войцех', 'Мориба', 'Лосады', 'Мартин', 'Бальде', 'Де Йонг', 'Мартинес', 'Лаго', 'Каррейра', 'Ольмо', 'Ямал', 'Рафинья', 'Ламин', 'Лопес', 'Мингеса'])

In [10]:
import requests
limit = 10
offset = 0
results = []
i = 0
while i < 10:
  news_items_raw = requests.get(url=f"http://0.0.0.0:8000/news?limit={limit}&offset={offset}")
  news_items = news_items_raw.json()
  offset += limit
  if not news_items:
    break
  
  for obj in news_items:
    if len(obj["content"].replace(" ", "")) < 50:
      continue
    if obj["lang"] == 'ru':
      results.append(f'{obj["source_url"]}\n{obj["content"]}\n\n')
  i += 1


In [11]:
f = "".join(results)
print(f)

https://tass.ru/mezhdunarodnaya-panorama/23734637
Это является частью плана президента США остановить критику в свой адрес, отметила демократ
НЬЮ-ЙОРК, 21 апреля. /ТАСС/. Сокращение финансирования Гарвардского университета является частью "возмутительного" плана президента США Дональда Трампа остановить критику в свой адрес. Такое мнение высказала губернатор - демократ штата Массачусетс Мора Хили.
"Это возмутительно, но это продолжение той стратегии, которую продолжает использовать Дональд Трамп, чтобы заставить критиков замолчать. <...> Теперь он нападает на колледжи и университеты, используя любые методы, чтобы попытаться закрыть их, заставить молчать. <...> В этом нет ничего хорошего", - заявила она в интервью телеканалу CBS.
По словам Хили, уменьшение финансирования учебных заведений негативно повлияет на исследования в сфере разработки лекарств и методов лечения рака. Более того, она утверждает, что сокращения приведут повлияют на позиции США в области медицинских исследований.
Ра

In [12]:
len(f)

150625